### HHS Hospital Data

An analysis of HHS hospital data, by @latimes and @stiles.

**Questions?** [matt.stiles@latimes.com](matt.stiles@latimes.com) \\ 310.529.8749

### Import Python tools

In [1]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import shapely
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Fetch the LA Times cities metadata

In [2]:
metadata_df = pd.read_csv("input/city-metadata.csv", dtype={"fips": str})

In [3]:
metadata_df = metadata_df[["county", "fips", "population", "region"]]

### Fetch the latest url to download the HHS data

In [4]:
# Data dictionary: https://healthdata.gov/covid-19-reported-patient-impact-and-hospital-capacity-facility-data-dictionary
# https://healthdata.gov/dataset/covid-19-reported-patient-impact-and-hospital-capacity-facility

### Get the latest url from the HHS API and read in the latest dataframe

In [5]:
metadata = pd.read_json('https://healthdata.gov/api/3/action/package_show?id=d475cc4e-83cd-4c16-be57-9105f300e0bc&page=0').result[0]['resources']

In [6]:
src = pd.read_csv(metadata[0]['url'],\
                 dtype={'fips_code':str, 'zip':str}, infer_datetime_format=True, parse_dates=True)

In [7]:
# src = pd.read_csv('https://healthdata.gov/sites/default/files/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20201228.csv',\
# dtype={'fips_code':str, 'zip':str}, infer_datetime_format=True, parse_dates=True)

In [8]:
df = src.copy()

In [9]:
ca_data = pd.DataFrame(df.loc[df.state == "CA"])

In [10]:
ca_data["fips"] = ca_data["fips_code"].str[2:5]

In [11]:
ca_timeseries = pd.merge(
    ca_data, metadata_df, how="left", left_on="fips", right_on="fips"
)

### What's the most recent collection week?

In [12]:
ca_timeseries['collection_week'] = pd.to_datetime(ca_timeseries['collection_week'])

In [13]:
ca_timeseries['collection_week'].max()

Timestamp('2020-12-18 00:00:00')

### Get rid of the -999999 suppressed values

In [14]:
ca_timeseries = ca_timeseries.replace([-999999.0], [0])

---

## Calculate some hospitalization/covid rates

#### Calculate daily average of total staffed ICU beds

In [15]:
ca_timeseries["total_staffed_adult_icu_beds"] = round((
    ca_timeseries["total_staffed_adult_icu_beds_7_day_sum"]
    / ca_timeseries["total_staffed_adult_icu_beds_7_day_coverage"]
),0)

#### Calculate daily average of occupied ICU beds

In [16]:
ca_timeseries["total_occupied_adult_icu_beds"] = round((
    ca_timeseries["staffed_adult_icu_bed_occupancy_7_day_sum"]
    / ca_timeseries["staffed_adult_icu_bed_occupancy_7_day_coverage"]
),0)

#### Calculate daily average of COVID patients in the ICU

In [17]:
ca_timeseries["total_covid_icu_patients"] = round((
    ca_timeseries["staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_sum"]
    / ca_timeseries[
        "staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_coverage"
    ]
),0)

#### Number of ICU beds available

In [18]:
ca_timeseries["total_available_adult_icu_beds"] = round((
    ca_timeseries["total_staffed_adult_icu_beds"]
    - ca_timeseries["total_occupied_adult_icu_beds"]
),0)

#### Calculate daily ICU occupancy as percentage

In [19]:
ca_timeseries["pct_occupied_adult_icu_beds"] = round((
    ca_timeseries["total_occupied_adult_icu_beds"]
    / ca_timeseries["total_staffed_adult_icu_beds"]
),2)

#### If a hospital reports 0 staffed adult ICU beds, drop them from the dataframe

In [20]:
filtered_timeseries = ca_timeseries[
    (ca_timeseries.total_staffed_adult_icu_beds_7_day_sum.notnull())
    & (ca_timeseries.total_staffed_adult_icu_beds_7_day_sum != 0)
].sort_values("total_staffed_adult_icu_beds_7_day_sum")

#### How many patients?

In [21]:
ca_timeseries["all_patients"] = round((
    ca_timeseries["all_adult_hospital_inpatient_bed_occupied_7_day_sum"]
    / ca_timeseries["all_adult_hospital_inpatient_bed_occupied_7_day_coverage"]
),0)

#### How many Covid patients

In [22]:
ca_timeseries["all_covid_patients"] = round((
    ca_timeseries["total_adult_patients_hospitalized_confirmed_covid_7_day_sum"]
    / ca_timeseries["total_adult_patients_hospitalized_confirmed_covid_7_day_coverage"]
),0)

#### What's the rate of Covid patients?

In [23]:
ca_timeseries["covid_patients_share"] = round((
    ca_timeseries["all_covid_patients"]
    / ca_timeseries["all_patients"]
),2)

In [24]:
ca_timeseries = ca_timeseries[ca_timeseries['all_patients'] > 0]

---

### Trim to the columns we want

In [25]:
trimmed_timeseries = ca_timeseries[
    [
        "hospital_name",
        "hospital_subtype",
        "ccn",
        "collection_week",
        "county",
        "fips",
        "total_covid_icu_patients",
        "total_available_adult_icu_beds",
        "pct_occupied_adult_icu_beds",
        "all_covid_patients",
        "all_patients",
        "covid_patients_share",
    ]
].rename(
    columns={
        "hospital_name": "hospital",
        "collection_week": "week",
    }
)

---

### Filter the dataframe for Los Angeles County facilities

In [26]:
la = trimmed_timeseries[trimmed_timeseries['fips'] == '037']

### Filter the California dataframe to the most recent collection week

In [27]:
current_ca = trimmed_timeseries[trimmed_timeseries['week'] == trimmed_timeseries['week'].max()].sort_values('covid_patients_share', ascending=False)

In [28]:
current_la = la[la['week'] == la['week'].max()].sort_values('covid_patients_share', ascending=False)

### Which CA hospitals have the greatest share of covid patients? 

In [29]:
current_ca.sort_values('covid_patients_share', ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,all_covid_patients,all_patients,covid_patients_share
235,BARSTOW COMMUNITY HOSPITAL,Short Term,050298,2020-12-18,San Bernardino,071,15.0,-2.0,1.22,47.0,42.0,1.12
230,DESERT VALLEY HOSPITAL,Short Term,050709,2020-12-18,San Bernardino,071,21.0,0.0,1.00,103.0,123.0,0.84
314,MAYERS MEMORIAL HOSPITAL,Critical Access Hospitals,051305,2020-12-18,Shasta,089,0.0,0.0,NaN,10.0,12.0,0.83
242,KAISER FOUNDATION HOSPITAL FONTANA,Short Term,050140,2020-12-18,San Bernardino,071,70.0,7.0,0.93,354.0,435.0,0.81
120,KAISER FOUNDATION HOSPITAL - DOWNEY,Short Term,050139,2020-12-18,Los Angeles,037,31.0,1.0,0.98,186.0,231.0,0.81
241,MOUNTAINS COMMUNITY HOSPITAL,Critical Access Hospitals,051312,2020-12-18,San Bernardino,071,0.0,0.0,NaN,4.0,5.0,0.80
42,EL CENTRO REGIONAL MEDICAL CENTER,Short Term,050045,2020-12-18,Imperial,025,30.0,2.0,0.94,132.0,172.0,0.77
78,COMMUNITY HOSPITAL OF HUNTINGTON PARK,Short Term,050091,2020-12-18,Los Angeles,037,9.0,0.0,1.00,38.0,50.0,0.76
162,GEORGE L MEE MEMORIAL HOSPITAL,Critical Access Hospitals,051336,2020-12-18,Monterey,053,0.0,0.0,NaN,6.0,8.0,0.75
193,GARDEN GROVE HOSPITAL & MEDICAL CENTER,Short Term,050230,2020-12-18,Orange,059,10.0,0.0,1.00,49.0,66.0,0.74


### Which facilities in LA have the greatest share of covid patients? 

In [30]:
current_la[(current_la['hospital_subtype'] != 'Long Term') & \
          (current_la['total_covid_icu_patients'] > 0)]\
.sort_values('covid_patients_share', ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,all_covid_patients,all_patients,covid_patients_share
120,KAISER FOUNDATION HOSPITAL - DOWNEY,Short Term,050139,2020-12-18,Los Angeles,037,31.0,1.0,0.98,186.0,231.0,0.81
78,COMMUNITY HOSPITAL OF HUNTINGTON PARK,Short Term,050091,2020-12-18,Los Angeles,037,9.0,0.0,1.00,38.0,50.0,0.76
73,KAISER FOUNDATION HOSPITAL - BALDWIN PARK,Short Term,050723,2020-12-18,Los Angeles,037,21.0,0.0,1.00,136.0,185.0,0.74
64,KAISER FOUNDATION HOSPITAL - PANORAMA CITY,Short Term,050137,2020-12-18,Los Angeles,037,21.0,2.0,0.93,116.0,173.0,0.67
71,EAST LOS ANGELES DOCTORS HOSPITAL,Short Term,050641,2020-12-18,Los Angeles,037,8.0,2.0,0.80,27.0,41.0,0.66
127,WHITE MEMORIAL MEDICAL CENTER,Short Term,050103,2020-12-18,Los Angeles,037,18.0,1.0,0.96,155.0,244.0,0.64
118,ANTELOPE VALLEY HOSPITAL,Short Term,050056,2020-12-18,Los Angeles,037,20.0,1.0,0.97,134.0,210.0,0.64
136,EMANATE HEALTH INTER-COMMUNITY HOSPITAL,Short Term,050382,2020-12-18,Los Angeles,037,22.0,3.0,0.92,151.0,239.0,0.63
99,"MARTIN LUTHER KING, JR. COMMUNITY HOSPITAL",Short Term,050779,2020-12-18,Los Angeles,037,12.0,1.0,0.96,127.0,202.0,0.63
124,GLENDALE MEM HOSPITAL & HLTH CENTER,Short Term,050058,2020-12-18,Los Angeles,037,20.0,0.0,1.00,100.0,161.0,0.62


### Which facilities in LA have the highest ICU capacity rates? 

In [31]:
current_la[(current_la['total_covid_icu_patients'] > 0)]\
.sort_values('pct_occupied_adult_icu_beds', ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,all_covid_patients,all_patients,covid_patients_share
96,CEDARS-SINAI MEDICAL CENTER,Short Term,050625,2020-12-18,Los Angeles,037,49.0,-20.0,1.19,207.0,821.0,0.25
119,KINDRED HOSPITAL SOUTH BAY,Long Term,052050,2020-12-18,Los Angeles,037,3.0,-1.0,1.17,22.0,51.0,0.43
140,EMANATE HEALTH FOOTHILL PRESBYTERIAN HOSPITAL,Short Term,050597,2020-12-18,Los Angeles,037,12.0,0.0,1.00,25.0,73.0,0.34
95,MONTEREY PARK HOSPITAL,Short Term,050736,2020-12-18,Los Angeles,037,5.0,0.0,1.00,43.0,71.0,0.61
113,WHITTIER HOSPITAL MEDICAL CENTER,Short Term,050735,2020-12-18,Los Angeles,037,16.0,0.0,1.00,55.0,121.0,0.45
114,CENTINELA HOSPITAL MEDICAL CENTER,Short Term,050739,2020-12-18,Los Angeles,037,35.0,0.0,1.00,94.0,218.0,0.43
122,PROVIDENCE LITTLE CO OF MARY MED CTR SAN PEDRO,Short Term,050078,2020-12-18,Los Angeles,037,11.0,0.0,1.00,46.0,95.0,0.48
135,PROVIDENCE HOLY CROSS MEDICAL CENTER,Short Term,050278,2020-12-18,Los Angeles,037,22.0,0.0,1.00,139.0,286.0,0.49
70,SAN DIMAS COMMUNITY HOSPITAL,Short Term,050588,2020-12-18,Los Angeles,037,11.0,0.0,1.00,30.0,60.0,0.50
137,PACIFICA HOSPITAL OF THE VALLEY,Short Term,050378,2020-12-18,Los Angeles,037,6.0,0.0,1.00,28.0,56.0,0.50


### How many LA County hospitals (with Covid patients) are in our dataframe? 

In [32]:
len(current_la[(current_la['total_covid_icu_patients'] > 0)])

73

### How many have 'high' ICU occupancy rates? 

In [33]:
len(current_la[current_la['pct_occupied_adult_icu_beds'] > .95 ])

32

### Let's look at Bay Area hospitals, too

In [34]:
bayarea = ['Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco', 'San Mateo', 'Santa Clara', 'Solano', 'Sonoma']

### Top 10 Bay Area hospotals by Covid share? 

In [35]:
current_ca[current_ca['county'].isin(bayarea)].sort_values('covid_patients_share', ascending=False).head(10)

,hospital,hospital_subtype,ccn,week,county,fips,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,all_covid_patients,all_patients,covid_patients_share
11,ST ROSE HOSPITAL,Short Term,050002,2020-12-18,Alameda,001,7.0,5.0,0.67,30.0,63.0,0.48
304,KAISER FOUNDATION HOSPITAL-SAN JOSE,Short Term,050604,2020-12-18,Santa Clara,085,12.0,2.0,0.89,72.0,154.0,0.47
293,SAN MATEO MEDICAL CENTER,Short Term,050113,2020-12-18,San Mateo,081,6.0,0.0,1.00,18.0,39.0,0.46
1,KAISER FOUNDATION HOSPITAL - FREMONT,Short Term,050512,2020-12-18,Alameda,001,7.0,1.0,0.90,31.0,74.0,0.42
289,KAISER FOUNDATION HOSPITAL - SOUTH SAN FRANCISCO,Short Term,050070,2020-12-18,San Mateo,081,6.0,1.0,0.90,38.0,94.0,0.40
305,REGIONAL MEDICAL CENTER OF SAN JOSE,Short Term,050125,2020-12-18,Santa Clara,085,20.0,0.0,1.00,90.0,226.0,0.40
20,KAISER FOUNDATION HOSPITAL - ANTIOCH,Short Term,050760,2020-12-18,Contra Costa,013,4.0,1.0,0.88,34.0,100.0,0.34
290,SETON MEDICAL CENTER,Short Term,050289,2020-12-18,San Mateo,081,4.0,0.0,1.00,24.0,71.0,0.34
303,KAISER FOUNDATION HOSPITAL-SANTA CLARA,Short Term,050071,2020-12-18,Santa Clara,085,12.0,2.0,0.93,79.0,239.0,0.33
19,SAN RAMON REGIONAL MEDICAL CENTER,Short Term,050689,2020-12-18,Contra Costa,013,4.0,6.0,0.50,18.0,55.0,0.33


---

## Get medical facilities' geographic data

In [36]:
# From HHS: https://maps3.arcgisonline.com/arcgis/rest/services/A-16/HHS_IOM_Health_Resources/MapServer

In [37]:
hospitals = gpd.read_file('/Users/mhustiles/data/github/AGStoShapefile/backupdir/HHS_Hospital/HHS_Hospital_1608139617293.geojson')

In [38]:
medical_centers = gpd.read_file('/Users/mhustiles/data/github/AGStoShapefile/backupdir/HHS_Medical_Center/HHS_Medical_Center_1608139616289.geojson')

### Clean up the column names so we can merge the facilities dataframes

In [39]:
facilities = [hospitals, medical_centers]

In [40]:
hospitals.rename(columns={'Name_new':'name', 'Address_1':'address', 'City':'city', 'State_1':'state', 'ZipCode':'zipcode',\
                               'PhoneNum':'phone', 'County_Nam':'county', 'Provider_N':'provider_id', 'Hospital_T':'type', 'Hospital_O':'operation',\
                               'Emergency_':'emergency'}, inplace=True)

In [41]:
medical_centers.rename(columns={'Hospital_N':'name', 'Address1':'address', 'City_1':'city', 'State_1':'state', 'ZipCode':'zipcode',\
                               'PhoneNum':'phone', 'County_Nam':'county', 'Provider_N':'provider_id', 'Hospital_T':'type', 'Hospital_O':'operation',\
                               'Emergency_':'emergency'}, inplace=True)

### Concatenate the different facility types into one dataframe, and also filter that just to CA

In [42]:
all_medical_geo = pd.concat(facilities)

In [43]:
all_medical_geo_ca = all_medical_geo[all_medical_geo['state'] == 'CA']

### We might be better off using HHS' own locations data with CCNs

In [44]:
locations = pd.read_csv('input/hospital_locations.csv', dtype={'latitude':float, 'longitude':float, 'CCN':str})

In [45]:
locations.columns = locations.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

### Even though it has the wrong location for White Memorial. Let's fix. 

In [46]:
locations.loc[(locations.facility_name == 'WHITE MEMORIAL MEDICAL CENTER'),'longitude']=-118.2176219

In [47]:
locations.loc[(locations.facility_name == 'WHITE MEMORIAL MEDICAL CENTER'),'latitude']=34.0493044

### Convert the lon/lat fields into a geodataframe

In [48]:
locations = gpd.GeoDataFrame(
    locations, geometry=gpd.points_from_xy(locations.longitude, locations.latitude))

### And then confine it to California

In [49]:
locationsca = gpd.GeoDataFrame(locations[locations['state'] == 'CA'])

### Make the CCN string match the hospital capacity data

In [50]:
locationsca['ccn'] = locationsca['ccn'].str.zfill(6)

---

### Merge with our CA medical facilities

In [51]:
current_ca_geo = locationsca.merge(current_ca, left_on='ccn', right_on='ccn')

In [52]:
len(current_ca_geo)

340

In [53]:
current_ca_geo.rename(columns={'county_x':'county', 'city_x':'city', 'address_x':'address', 'state_x':'state'}, inplace=True)

In [54]:
# current_ca_geo.drop(['county_y', 'hospital_subtype', 'provider_id', 'ccn', 'emergency', 'address', 'OBJECTID', 'state', 'name', 'phone', 'zipcode'], axis=1, inplace=True)

---

### Filter to just LA County hospitals

In [55]:
current_la_geo = current_ca_geo[current_ca_geo['fips'] == '037']

### How many facilities in California?

In [56]:
len(current_ca_geo)

340

### How many in L.A. County?

In [57]:
len(current_la_geo)

83

---

### Export hospital specific data for maps and tables

In [58]:
current_ca_geo.to_file('output/current_ca_geo.geojson', driver='GeoJSON')

In [59]:
current_ca_geo.to_csv('output/current_ca_geo.csv', index=False)

In [60]:
current_la_geo.to_file('output/current_la_geo.geojson', driver='GeoJSON')

In [61]:
current_la_geo.to_csv('output/current_la_geo.csv', index=False)

---

## Aggregate

### What's the average share of Covid patients in hospitals statewide?

In [62]:
state = current_ca.groupby(['week']).agg({'all_covid_patients':'sum', 'all_patients':'sum'}).reset_index()

In [63]:
state['covid_patients_share'] = round((
    state["all_covid_patients"]
    / state["all_patients"]
),2)

In [64]:
state.head()

,week,all_covid_patients,all_patients,covid_patients_share
0,2020-12-18,17545.0,46742.0,0.38


### Covid patients by county

In [65]:
counties = trimmed_timeseries.groupby(['county', 'week']).agg({'all_covid_patients':'sum', 'all_patients':'sum'}).reset_index()

In [66]:
counties['covid_patients_share'] = round((
    counties["all_covid_patients"]
    / counties["all_patients"]
),2)

#### Which counties have the most Covid patients (in most recent week)?

In [67]:
counties[counties['week'] == counties['week'].max()].sort_values('all_covid_patients', ascending=False).head(5)

,county,week,all_covid_patients,all_patients,covid_patients_share
373,Los Angeles,2020-12-18,6167.0,14342.0,0.43
604,Orange,2020-12-18,1718.0,3951.0,0.43
729,San Bernardino,2020-12-18,1539.0,2878.0,0.53
750,San Diego,2020-12-18,1293.0,3872.0,0.33
667,Riverside,2020-12-18,1267.0,2633.0,0.48


#### Which counties have the highest percentage of Covid patients (in most recent week)?

In [68]:
counties[counties['week'] == counties['week'].max()].sort_values('covid_patients_share', ascending=False).head(5)

,county,week,all_covid_patients,all_patients,covid_patients_share
247,Imperial,2020-12-18,194.0,263.0,0.74
268,Inyo,2020-12-18,8.0,11.0,0.73
310,Kings,2020-12-18,89.0,143.0,0.62
394,Madera,2020-12-18,32.0,54.0,0.59
83,Calaveras,2020-12-18,7.0,12.0,0.58


### How has this changed over time? 

In [69]:
counties_grouped = counties.groupby(['county', 'week']).agg({'all_covid_patients':'sum', 'covid_patients_share':'sum'}).reset_index()

In [70]:
socal = ['Los Angeles', 'Orange', 'Ventura', 'San Bernardino', 'Riverside']

In [71]:
la_grouped = counties_grouped[counties_grouped['county'] == 'Los Angeles']

In [72]:
la_grouped.tail()

,county,week,all_covid_patients,covid_patients_share
369,Los Angeles,2020-11-20,1675.0,0.14
370,Los Angeles,2020-11-27,2422.0,0.19
371,Los Angeles,2020-12-04,3308.0,0.25
372,Los Angeles,2020-12-11,4671.0,0.34
373,Los Angeles,2020-12-18,6167.0,0.43


In [73]:
alt.Chart(la_grouped).mark_area().encode(
    x=alt.X('week:T', axis=alt.Axis(format='', title=' ', tickCount=3)),
    y = alt.Y('all_covid_patients', title='Covid patients in LA County hospitals', axis=alt.Axis(format='', tickCount=6)),
).properties(width=500, height=500)

alt.Chart(...)

### Just southern California counties? 

In [74]:
socal_grouped = counties_grouped[counties_grouped['county'].isin(socal)].groupby(['week']).agg({'all_covid_patients':'sum', 'covid_patients_share':'mean'}).reset_index()

In [75]:
socal_grouped.tail()

,week,all_covid_patients,covid_patients_share
16,2020-11-20,3173.0,0.158
17,2020-11-27,4546.0,0.214
18,2020-12-04,6086.0,0.266
19,2020-12-11,8483.0,0.358
20,2020-12-18,10978.0,0.442


In [76]:
alt.Chart(socal_grouped).mark_area().encode(
    x=alt.X('week:T', axis=alt.Axis(format='', title=' ', tickCount=3)),
    y = alt.Y('covid_patients_share', title=' ', axis=alt.Axis(format='%', title='Share of all patients with Covid in SoCal', tickCount=6)),
).properties(width=500, height=500)

alt.Chart(...)

### Bay Area? 

In [77]:
bayarea_grouped = counties_grouped[counties_grouped['county'].isin(bayarea)].groupby(['week']).agg({'all_covid_patients':'sum', 'covid_patients_share':'mean'}).reset_index()

In [78]:
bayarea_grouped.tail()

,week,all_covid_patients,covid_patients_share
16,2020-11-20,603.0,0.081111
17,2020-11-27,815.0,0.111111
18,2020-12-04,1050.0,0.132222
19,2020-12-11,1383.0,0.174444
20,2020-12-18,1675.0,0.204444


In [79]:
alt.Chart(bayarea_grouped).mark_area().encode(
    x=alt.X('week:T', axis=alt.Axis(format='', title=' ', tickCount=3)),
    y = alt.Y('covid_patients_share', title=' ', axis=alt.Axis(format='%', title='Share of all patients with Covid in Bay Area', tickCount=6)),
).properties(width=500, height=500)

alt.Chart(...)